In [2]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import dagshub

dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing as satyajeetrai007

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [3]:
# Step 1: Isolate features and target, dropping rows with missing labels
df = df.dropna(subset=['category'])
X = df['clean_comment']
y = df['category']

# Step 2: Perform the train-test split FIRST to prevent any data leakage
# The 'stratify' parameter ensures class proportions are the same in train and test sets.
X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Fit the TF-IDF vectorizer ONLY on the training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train_text)

# Now, use the fitted vectorizer to transform the test data
X_test_vec = vectorizer.transform(X_test_text)

# Step 4: Apply SMOTE to the TRAINING data ONLY to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# The test data (X_test_vec, y_test) is NEVER resampled.

# --- MLflow and Optuna Functions (no changes needed) ---

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams_Corrected")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        
        model_filename = f"{model_name}_model.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_filename, "model")

# Step 5: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    # Train on resampled training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Evaluate on the original, untouched test data
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 6: Run Optuna for Logistic Regression
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)
    
    # Log the best model using the correctly processed datasets
    log_mlflow("LogisticRegression", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment
run_optuna_experiment()

[I 2025-08-27 20:21:00,478] A new study created in memory with name: no-name-89b2a1be-38c5-4158-ae50-6982fac44feb
c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-08-27 20:21:01,940] Trial 0 finished with value: 0.6941647123741206 and parameters: {'C': 0.012322260322439056, 'penalty': 'l2'}. Best is trial 0 with value: 0.6941647123741206.
c:\Users\satya\OneDrive\Desktop\youtube-comment-senitment-analysis\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver w

🏃 View run LogisticRegression_SMOTE_TFIDF_Trigrams_Corrected at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/41e7d5b473d7482fa08bae898657063b
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
